In [ ]:
import pandas as pd
import json
import urllib
import requests as rq
import base64
import httplib2

import ast, json
import pandas as pd
import warnings
from pandas import json_normalize

import psycopg2   
from psycopg2 import Error
import geopandas as gpd
import numpy as np

from tqdm import tqdm
from shapely.geometry import LineString, Point
from shapely.wkt import dumps, loads

from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

listings = pd.DataFrame()

In [ ]:
def get_oauth_token():
    http_obj = httplib2.Http()
    url = "https://api.idealista.com/oauth/token"
    apikey= urllib.parse.quote_plus('__apikey__')
    secret= urllib.parse.quote_plus('__secret__')
    param = str(apikey + ':' + secret).encode()
    auth = base64.b64encode(param)
    body = {'grant_type':'client_credentials'}
    headers = {'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8','Authorization' : 'Basic ' + auth.decode()}
    resp, content = http_obj.request(url,method='POST',headers=headers, body=urllib.parse.urlencode(body))
    return eval(content.decode())['access_token']

In [ ]:
def search_api(token, type_listing, page, operation):
    http_obj = httplib2.Http()
    center = "38.72623322072787,-9.14011001586914"
    url = "https://api.idealista.com/3.5/pt/search?center="+center+"&operation="+operation+"&propertyType="+type_listing+"&distance=5000&maxItems=50&numPage="+str(page)
    headers = {'Authorization' : 'Bearer ' + token}
    resp, content = http_obj.request(url,method='POST',headers=headers)
    
    return content.decode()

In [ ]:
rows = json.loads(search_api(get_oauth_token(), 'garages', 2, 'rent'))

In [ ]:
for i in range(len(rows['elementList'])):

    df = pd.DataFrame.from_dict(rows['elementList'][i], orient='index').transpose()
    df = pd.concat([df, json_normalize(df['detailedType'])], axis=1).drop(['detailedType'], axis=1)
    df = pd.concat([df, json_normalize(df['suggestedTexts'])], axis=1).drop(['suggestedTexts'], axis=1)
    listings = listings.append(df)
    


In [ ]:
listings = listings.drop_duplicates(subset=["propertyCode"], keep='first')

In [ ]:
table_name = ""
database = ""
user = "postgres"
password = "postgres"
port = str(5433)
host = "localhost"

engine = create_engine("postgresql://"+user+":"+password+"@"+host+":"+port+"/"+database)

def generate_geometry(x):
    return loads(dumps(Point(x['longitude'], x['latitude']), rounding_precision=10))

In [ ]:
listings['geometry'] = listings.apply(generate_geometry, axis=1)
listings = listings[listings['geometry'].notna()]

gpd_insta = gpd.GeoDataFrame(listings, geometry='geometry')
gpd_insta.set_crs(epsg=4326, inplace=True, allow_override=True)
gpd_insta = gpd_insta.__round__(10)

## Create table in postgis and insert data
gpd.GeoDataFrame.to_postgis(gpd_insta, name = table_name, con = engine, schema = 'public', if_exists = 'append')